In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, project_root)

import pandas as pd
from src.statistical_testing import perform_wilcoxon

pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv('../reports/raw_metrics.csv')
df.head()

,Dataset,Sensitive Attribute,Mitigation,accuracy,f1_score,SPD,DI,EOD,AOD,Pipeline
0,compas,race,baseline,0.680527,0.606319,-0.196925,0.539225,-0.221744,-0.167383,base
1,compas,race,baseline,0.663895,0.589331,-0.188922,0.565712,-0.249795,-0.169028,base
2,compas,race,baseline,0.661123,0.581694,-0.110155,0.722155,-0.121066,-0.093272,base
3,compas,race,baseline,0.683992,0.608247,-0.181815,0.565107,-0.152574,-0.148753,base
4,compas,race,baseline,0.686071,0.605057,-0.138029,0.647506,-0.168502,-0.109302,base


In [3]:
wilcoxon_results = perform_wilcoxon(df)

Number of (Dataset, Attribute, Method) combos: 16
Performed 96 Wilcoxon tests (6 metrics × 16 combos = 96).


/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
/opt/anaconda3/lib/python3.11/site-packages/scipy/stats/_morestats.py:4088: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [17]:
wilcoxon_results[37:40]

,Dataset,Attribute,Pipeline,Method,Metric,W,p-value,p_adj,significant,significance
37,adult,sex,inprocessing,meta fair classifier,f1_score,32.0,0.000,0.001,True,**
38,adult,sex,inprocessing,prejudice remover,f1_score,81.0,0.027,0.165,False,
39,adult,sex,preprocessing,reweighing,f1_score,0.0,0.000,0.000,True,***


In [5]:
results_nested = {}
for method, sub in wilcoxon_results.groupby('Method'):
    results_nested[method] = {}
    for (ds, attr), grp in sub.groupby(['Dataset','Attribute']):
        # make a small DF of metrics × [W, p-value, p_adj]
        results_nested[method][(ds,attr)] = (
            grp.set_index('Metric')[['W','p-value','p_adj']]
        )

# e.g. access Reweighing → (Adult_income, race)
results_nested['reweighing'][('compas','sex')]

,W,p-value,p_adj
Metric,,,
accuracy,41.5,0.001,0.004
f1_score,22.0,0.000,0.000
SPD,0.0,0.000,0.000
DI,0.0,0.000,0.000
EOD,0.0,0.000,0.000
AOD,0.0,0.000,0.000
